# Imputation of missing values

This is a component for imputing missing values using mean, median or most frequent. It makes use of an implementation from [Scikit-learn](https://scikit-learn.org/stable/modules/generated/sklearn.impute.SimpleImputer.html#sklearn.impute.SimpleImputer). 
<br>
Scikit-learn is an open source machine learning library that supports supervised and unsupervised learning. It also provides various tools for model fitting, data preprocessing, model selection and evaluation, and many other utilities.

This notebook shows:
- how to use SDK to load the dataset and save a model.
- how to receive parameters from the platform.

## Declare parameters
Components may declare (and use) these default parameters:
- dataset
- target
- experiment_id
- operator_id

Use these parameters to load/save datasets, models, metrics, and figures with the [PlatIAgro SDK](https://platiagro.github.io/sdk/).

You may also declare custom parameters to set inside an experiment.

In [ ]:
dataset = "iris" #@param {type:"string"}
target = "Species" #@param {type:"string"}
experiment_id = "b48b03b1-5581-4e1d-a429-f31a34f78e1c" #@param {type:"string"}
operator_id = "76f521c2-ff17-4850-944d-09188eeb27c1" #@param {type:"string"}
strategy = "most_frequent" #@param {type:"string","label":"Estratégia de atribuição de valores","description":"mean - apenas para atributos numéricos<br>median - apenas para atributos numéricos<br>most_frequent - em atributos numéricos e categóricos"}

## Load dataset

Import and put the whole dataset in a pandas.DataFrame.

In [ ]:
from platiagro import load_dataset

df = load_dataset(name=dataset)
X = df.drop(target, axis=1).to_numpy()
y = df[target].to_numpy()

In [ ]:
import numpy as np

columns = df.columns.to_numpy()
target_index = np.argwhere(columns == target)
columns = np.delete(columns, target_index)

## Load metadata about the dataset
For example, below we get the feature type for each column in the dataset. (eg. categorical, numerical, or datetime)

In [ ]:
from platiagro import stat_dataset
from platiagro.featuretypes import infer_featuretypes

try:
    metadata = stat_dataset(name=dataset)
    featuretypes = metadata["featuretypes"]
except KeyError:
    featuretypes = infer_featuretypes(df)

featuretypes = np.array(featuretypes)
featuretypes = np.delete(featuretypes, target_index)

## Fit a model using sklearn.impute.SimpleImputer

In [ ]:
import numpy as np
import pandas as pd
from platiagro.featuretypes import NUMERICAL
from sklearn.impute import SimpleImputer

# Replace None with np.nan
df.fillna(value=pd.np.nan, inplace=True)

# Put data back in a numpy.ndarray
X = df.drop(target, axis=1).to_numpy()

numerical_indexes = (featuretypes == NUMERICAL)

estimator = SimpleImputer(missing_values=np.nan, strategy=strategy)

if strategy == "mean" or strategy == "median":
    X[:, numerical_indexes] = estimator.fit_transform(X[:, numerical_indexes])
    columns = columns[~pd.isna(estimator.statistics_)]
    df = pd.DataFrame(data=X, columns=columns[~pd.isna(estimator.statistics_)])
    df[target] = pd.Series(y)
elif strategy == "most_frequent":
    X = estimator.fit_transform(X)
    df = pd.DataFrame(data=X, columns=columns[~pd.isna(estimator.statistics_)])
    df[target] = pd.Series(y)

## Save dataset

Stores the transformed dataset in a object storage.<br>

In [ ]:
from platiagro import save_dataset

save_dataset(name=dataset, df=df)

## Save model

Stores the model artifacts in a object storage.<br>
It will make the model available for future deployments.

In [ ]:
from platiagro import save_model

save_model(experiment_id=experiment_id,
           model={"estimator": estimator,
                  "columns": columns,
                  "strategy": strategy,
                  "numerical_indexes": numerical_indexes})